In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float,clean_data     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'height'
field1='Length'
field2='Length'
fields = [field1,field2 ]
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'

In [3]:
df,custom_field_df=parameters('90','height',field1, )

1194


In [4]:
del df['bucket_id']
del df['product_id']
del df['image_url']
del df['value']
del df['custom_fields']

In [5]:
df_cur=df
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3Le?n?g?t?h?\b)|()'''
df_cur['matches_length'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'''(?i)((?<!up.to.)(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}He?i?g?h?t?(?! Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Wi?d?t?h?\b)|()'''
df_cur['matches_width'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'''(?i)((?<!\d)(?<!\w )(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Diam?e?t?e?r?\b)|(.{0,20}core.?dia.{0,20})|(\d.{0,5}Core)|()'''
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# df_cur['new_matches'] = df_cur['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'''(?i)((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x?\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)?\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|()'''
df_cur['blank_product'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# Height

In [6]:
from natsort import natsorted
regex_pattern=r'''(?i)(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}He?i?g?h?t?|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# Filter out the SKUs that dont have numbers
pat_number=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(pat_number, str(x)))
lengthNew=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]
del lengthNew['new_matches']

lengthNew['matches']=lengthNew['matches'].apply(lambda x: natsorted(x)).apply(lambda x: sorted(x))
lengthNew['matchez']=lengthNew['matches'].apply(lambda x: re.sub(r'''(?i)((?:\\?\"|\”|inc?h?|''|�)\W{0,2}Core)''',' in',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))


curate(lengthNew, 'matchez', formatted_attribute)

Repeated External IDs: []


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, matches_length, matches_height, matches_width, matches_diameter, blank_product, blank_long, matches, matchez, Q:height]
Index: []

In [7]:
matchesnew=lengthNew[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()
rounding(lengthNew, 'matchez','a-eghj-mo-su-z')

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, matches_length, matches_height, matches_width, matches_diameter, blank_product, blank_long, matches, matchez, Q:height, rounding]
Index: []

In [8]:
# print(len(df_cur[df_cur['matches_height'].astype(str)=='[]']))
# df_cur[df_cur['matches_height'].astype(str)=='[]'][0:500]

In [9]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
#           'customer_name':customer_name
         }

curation_col = f'Q:{attribute}'

import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(df['custom_fields'])
df_cur = pd.concat([df.drop('custom_fields', axis=1)], axis = 1)
print(len(df_cur))

# df_custom=df_cur[(df_cur[field1].astype(str)!='[]')&(df_cur[field1].astype(str)!='nan')&(df_cur[field1].astype(str)!='Adjustable')&(df_cur[field1].astype(str)!='NaN')]
# df_cur=df_cur[(df_cur[field1].astype(str)=='[]')|(df_cur[field1].astype(str)=='nan')|(df_cur[field1].astype(str)=='Adjustable')|(df_cur[field1].astype(str)=='NaN')]

1194


In [10]:
# df_custom['match']=df_custom[field1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).apply(lambda x: re.sub(r'\"',' in',str(x))).apply(lambda x: re.sub(r"\'",'"',str(x))).apply(lambda x: re.sub(r"Up to ",'"',str(x)))
# rounds(df_custom, 'match')
# df_custom[curation_col]=df_custom['match']
# matchcust=df_custom[['external_id',curation_col]]
# print(len(matchcust))
# rounding(matchcust, curation_col)

In [11]:
# regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
# df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)((?<!\-)(?<!\d)(?<!\,)(?<!\w)\d+(?:\.\d+|.\d+\/\d+)?[^\w\d]{0,3}He?i?g?h?t?(?!\-back)(?!\sLabel)(?!.Adjustment)(?!.Adjustable)\b)|((?<!\.)(?<!up.to.)(?<!\,)(?<!\w)(?<![\w])(?<!\/)(?<!\/)(?<!\-)\d+(?!\.\d)(?<!\d\\\"\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!Adjustable)(?!\sLabel)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
# regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
# df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
# regex_pattern_diameter=r'(?i)((?<!x )\d+\.\d+.?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )(?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b(?! x))|((?<!x )(?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b(?! x))|()'
# df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df_cur['blank_matches'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_matches_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [12]:
from natsort import natsorted
regex_pattern=r'''(?i)((?<!\-)(?<!\d)(?<!\,)(?<!\w)\d+(?:\.\d+|.\d+\/\d+)?[^\w\d]{0,3}He?i?g?h?t?(?!\-back)(?!\sLabel)(?!.Adjustment)(?!.Adjustable)\b)|((?<!\.)(?<!up.to.)(?<!\,)(?<!\w)(?<![\w])(?<!\/)(?<!\/)(?<!\-)\d+(?!\.\d)(?<!\d\\\"\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!Adjustable)(?!\sLabel)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_cur[df_cur['matches'].astype(str)!='[]']))

667


In [13]:
# del df_cur['bucket_id']
# del df_cur['product_id']
# del df_cur['image_url']
# del df_cur['value']
# # del df_cur['Height']

In [14]:
regex_pattern_blank=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
height_new=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]

height_new['matches']=height_new['matches'].apply(lambda x: natsorted(x))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r"(?<=\d)\'",' ft',str(x))).apply(lambda x: re.sub(r'(?i)height|"\s?high',' in',str(x))).apply(lambda x: re.sub(r'(?i)height|\\" H|"H|H',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
height_new['matchez']=height_new['matches'].apply(lambda x: re.sub(r"\'",' ft',str(x))).apply(lambda x: re.sub(r'ft in|ftin','ft',str(x)))
# height_new['matchez']=height_new['matchez'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

# height_new['matchez']=height_new['matchez'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# height_new['matchez']=height_new['matchez'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# height_new['matchea']=height_new['matchez'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
# del height_new['new_matches']

# height_new['matchea']=height_new['matchea'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# height_new['matchea']=height_new['matchea'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# height_new['matchea']=height_new['matchea'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
# height_new['matchea']=height_new['matchea'].apply(lambda x: re.sub(r"ft in",'ft',str(x)))
clean_data(height_new,'matchez',curation_col)
curate(height_new, 'matchez', attribute)

# height_new[curation_col]=height_new[curation_col].apply(lambda x: re.sub(r"\'",' ft',str(x))).apply(lambda x: re.sub(r'ft in|ftin','ft',str(x))).apply(lambda x: re.sub(r'\]','"]',str(x))).apply(lambda x: re.sub(r'(?<!"),','",',str(x)))
rounding(height_new, curation_col,'a-eghj-mo-su-z')

Repeated External IDs: []


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, matches_height, blank_matches, blank_matches_long, matches, new_matches, matchez, Q:height, rounding]
Index: []

In [15]:
# print(len(height_new))
# rounds(height_new, 'matchea')
# height_new[curation_col]=height_new['matchea']
# rounding(height_new, 'matchea')

In [16]:
matchesnew=height_new[['external_id',curation_col]]
# matchesnew[curation_col].explode().value_counts()
matchesnew[curation_col]=matchesnew[curation_col].apply(lambda x: re.sub(r'""','"',str(x)))

# NA Values

In [17]:
regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}['"].{0,2}[LWHD]?)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'''
regex_pattern_measure=r'(?i)(measure)|()'
df_na=df_cur
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))
print(len(df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')]))
df_na=df_na[(df_na['matches_height'].astype(str)=='[]')&(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')&(df_na['blank_matches_long'].astype(str)=='[]')]

34


In [18]:
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
# matchesna

In [19]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(msplit))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matchcustom))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchesna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matchcust))

In [20]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [ ]:
# looks_good('WBMason', attribute, pand, matchcustom)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, df, matchesna)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matchesnew)

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBYmatchesna - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
# looks_good('WBMason', attribute, df, matchcust)